In [1]:
import pandas as pd

# Carga el archivo CSV con '|' como delimitador
df = pd.read_csv('r.csv', sep='|')

# Muestra las primeras filas del DataFrame
df.head()


,Id,Categoria,Nombre,Valoracion,Dificultad,Num_comensales,Tiempo,Tipo,Link_receta,Num_comentarios,Num_reviews,Fecha_modificacion,Ingredientes
0,74138,Recetas de Aperitivos y tapas,Receta de Baleadas hondureñas,NaN,baja,2.0,10m,NaN,https://www.recetasgratis.net/receta-de-balead...,0,0,2020-04-06,"4 trotillas de harina,2 tazas de frijoles negr..."
1,74133,Recetas de Aperitivos y tapas,Receta de Tacos de carne molida,NaN,baja,4.0,30m,Plato principal,https://www.recetasgratis.net/receta-de-tacos-...,0,0,2020-03-27,"500 gramos de carne molida,1 papa mediana,2 cu..."
2,74154,Recetas de Aperitivos y tapas,Receta de Burritos norteños,NaN,muy baja,5.0,30m,Entrante,https://www.recetasgratis.net/receta-de-burrit...,0,0,2020-03-25,"10 tortillas de harina para burritos,500 gramo..."
3,74146,Recetas de Aperitivos y tapas,Receta de Croquetas de pollo caseras de la abuela,NaN,baja,4.0,45m,Entrante,https://www.recetasgratis.net/receta-de-croque...,0,0,2020-03-24,"1 pechuga de pollo cocida y troceada,1 pellizc..."
4,74081,Recetas de Aperitivos y tapas,Receta de Masa para empanadas fritas,NaN,baja,3.0,30m,Entrante,https://www.recetasgratis.net/receta-de-masa-p...,0,0,2020-03-03,"3 tazas de harina de trigo (420 gramos),1 pizc..."


In [2]:
df = df.drop("Valoracion", axis=1)
df = df.drop("Num_comentarios", axis=1)
df = df.drop("Num_reviews", axis=1)
df = df.drop("Fecha_modificacion", axis=1)
df = df.drop("Tipo", axis=1)

In [3]:
df.head()

,Id,Categoria,Nombre,Dificultad,Num_comensales,Tiempo,Link_receta,Ingredientes
0,74138,Recetas de Aperitivos y tapas,Receta de Baleadas hondureñas,baja,2.0,10m,https://www.recetasgratis.net/receta-de-balead...,"4 trotillas de harina,2 tazas de frijoles negr..."
1,74133,Recetas de Aperitivos y tapas,Receta de Tacos de carne molida,baja,4.0,30m,https://www.recetasgratis.net/receta-de-tacos-...,"500 gramos de carne molida,1 papa mediana,2 cu..."
2,74154,Recetas de Aperitivos y tapas,Receta de Burritos norteños,muy baja,5.0,30m,https://www.recetasgratis.net/receta-de-burrit...,"10 tortillas de harina para burritos,500 gramo..."
3,74146,Recetas de Aperitivos y tapas,Receta de Croquetas de pollo caseras de la abuela,baja,4.0,45m,https://www.recetasgratis.net/receta-de-croque...,"1 pechuga de pollo cocida y troceada,1 pellizc..."
4,74081,Recetas de Aperitivos y tapas,Receta de Masa para empanadas fritas,baja,3.0,30m,https://www.recetasgratis.net/receta-de-masa-p...,"3 tazas de harina de trigo (420 gramos),1 pizc..."


In [4]:
df = df.dropna(subset=["Ingredientes"])

In [5]:
df.isnull().sum()

Id                   0
Categoria            0
Nombre               0
Dificultad        7386
Num_comensales    9305
Tiempo            9307
Link_receta          0
Ingredientes         0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22150 entries, 0 to 22373
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              22150 non-null  int64  
 1   Categoria       22150 non-null  object 
 2   Nombre          22150 non-null  object 
 3   Dificultad      14764 non-null  object 
 4   Num_comensales  12845 non-null  float64
 5   Tiempo          12843 non-null  object 
 6   Link_receta     22150 non-null  object 
 7   Ingredientes    22150 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 1.5+ MB


In [7]:
df.shape

(22150, 8)

In [8]:
import pandas as pd
from antlr4 import *
from recipesLexer import recipesLexer
from recipesParser import recipesParser

In [9]:
# Definir una función para usar el parser de ANTLR4
def parse_ingredients(query):
    input_stream = InputStream(query)
    lexer = recipesLexer(input_stream)
    stream = CommonTokenStream(lexer)
    parser = recipesParser(stream)
    tree = parser.query()
    
    ingredients = []
    for child in tree.getChildren():
        if isinstance(child, recipesParser.IngredientContext):
            ingredients.append(child.getText().lower())
    
    return ingredients

In [10]:
# Función para filtrar recetas basadas en los ingredientes ingresados por el usuario
def filtrar_recetas(query, df):
    ingredientes = parse_ingredients(query)
    recetas_filtradas = df[df['Ingredientes'].apply(lambda x: all(ing in x.lower() for ing in ingredientes))]
    return recetas_filtradas

# Ejemplo de consulta ingresada por el usuario
consulta_usuario = "tomate, queso, albahaca, pollo"

In [11]:
# Filtrar recetas
if 'Ingredientes' in df.columns:
    recetas = filtrar_recetas(consulta_usuario, df)
    # Ajustar 'title' según el nombre de la columna en tu DataFrame, por ejemplo 'recipe_name'
    print('encontro')
else:
    print("La columna 'all_ingredients' no existe en el DataFrame. Asegúrate de haberla creado correctamente.")

encontro


In [12]:
df_recetas = pd.DataFrame(recetas, columns=['Nombre'])


In [13]:
df_rec = pd.DataFrame(recetas, columns=['Ingredientes'])


In [14]:
def buscar_recetas():
    query = "".join(selected_ingredients)
    print("QUERY: "+query)
    if 'Ingredientes' in df.columns:
        recetas = search_recipes(query, df)
        # Limpiar el Treeview
        for row in tree.get_children():
            tree.delete(row)
        if not recetas.empty:
            categorias = recetas['Categoria'].unique()
            for categoria in categorias:
                parent = tree.insert("", tk.END, values=(categoria, "", ""), open=True)
                recetas_categoria = recetas[recetas['Categoria'] == categoria]
                for index, row in recetas_categoria.iterrows():
                    tree.insert(parent, tk.END, values=("", row['Nombre'], row['Link_receta']))
            query = ""
        else:
            messagebox.showinfo("Sin resultados", "No se encontraron recetas con los ingredientes seleccionados.")
            query = ""
    else:
        messagebox.showerror("Error", "La columna 'Ingredientes' no existe en el DataFrame.")
        query = ""


In [31]:
import pandas as pd
from antlr4 import *
from recipesLexer import recipesLexer
from recipesParser import recipesParser
from recipesListener import recipesListener
import re
# Función para analizar la consulta usando ANTLR
def parse_query(query):
    input_stream = InputStream(query)
    lexer = recipesLexer(input_stream)
    stream = CommonTokenStream(lexer)
    parser = recipesParser(stream)
    tree = parser.query()
    listener = RecipeListener()
    walker = ParseTreeWalker()
    walker.walk(listener, tree)
    return listener

# Listener para extraer los ingredientes de la consulta
class RecipeListener(recipesListener):
    def __init__(self):
        self.ingredients = []
        self.logical_ops = []
        self.current_ingredient = []

    def exitIngredient(self, ctx: recipesParser.IngredientContext):
        ingredient = ctx.getText()
        self.current_ingredient.append(ingredient)

    def exitCondition(self, ctx: recipesParser.ConditionContext):
        if self.current_ingredient:
            self.ingredients.append(" ".join(self.current_ingredient))
            self.current_ingredient = []

    def exitQuery(self, ctx: recipesParser.QueryContext):
        for child in ctx.getChildren():
            if isinstance(child, TerminalNode) and child.getText() in [",", "|"]:
                self.logical_ops.append(child.getText())

# Función para buscar recetas en el DataFrame
def search_recipes(query, df):
    listener = parse_query(query)
    print(listener)
    ingredients = listener.ingredients
    logical_ops = listener.logical_ops
    print(ingredients)
    def contains_ingredient(ingredient, recipe_ingredients):
        # Agregar un espacio entre el número y la palabra si el número está seguido de letras
        ingredient_pattern = re.sub(r'(\d+)([a-zA-Z])', r'\1 \2', ingredient.lower())
        return bool(re.search(r'\b' + re.escape(ingredient_pattern) + r'\b', recipe_ingredients.lower()))

    # Inicialmente, seleccionamos todas las recetas
    filtered_df = df.copy()

    if not logical_ops:
        for ingredient in ingredients:
            filtered_df = filtered_df[filtered_df['Ingredientes'].apply(lambda x: contains_ingredient(ingredient, x))]
    else:
        result_mask = None
        for i, ingredient in enumerate(ingredients):
            ingredient_mask = df['Ingredientes'].apply(lambda x: contains_ingredient(ingredient, x))
            if result_mask is None:
                result_mask = ingredient_mask
            else:
                if logical_ops[i - 1] == ',':
                    result_mask = result_mask & ingredient_mask
                elif logical_ops[i - 1] == '|':
                    result_mask = result_mask | ingredient_mask

        filtered_df = df[result_mask]

    return filtered_df

In [32]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter import scrolledtext
import webbrowser

In [33]:
# Función para agregar ingredientes y desactivar botones
def add_ingredient(ingredient):
    global selected_ingredients
    if ingredient in [",", "|", "(", ")"]:
        selected_ingredients += ingredient  # Añadir directamente el separador
    else:
        if ingredient not in selected_ingredients:
            selected_ingredients.append(ingredient)
    update_ingredients_label()

In [34]:
def convert_logical_operators(query):
    query = query.replace('AND', ',')
    query = query.replace('OR', '|')
    return query

In [35]:
def update_ingredients_label():
    ingredients_label.config(text="Ingredientes seleccionados: " + "".join(selected_ingredients))

In [36]:
def on_treeview_click(event):
    item = tree.selection()
    if item:
        item = item[0]
        link = tree.item(item, "values")[2]
        webbrowser.open(link)

In [37]:
# Crear ventana principal
window = tk.Tk()
window.title("Aplicación de Recetas")

''

In [38]:
# Lista para almacenar los ingredientes seleccionados
selected_ingredients = []

In [39]:
# Lista de ingredientes disponibles
available_ingredients = ["Pollo", "Zanahoria", "Papa", "Tomate", "Cebolla","carne", ",", "|", "(", ")",
                        "1 ","2 ","3 ","4 ","5 ","6 ","7 ","8 ","9 ","0 "]

In [40]:
# Crear y posicionar los widgets de manera dinámica
for idx, ingredient in enumerate(available_ingredients):
    button = tk.Button(window, text=ingredient, command=lambda ingr=ingredient: add_ingredient(ingr))
    button.grid(row=0, column=idx, padx=5, pady=5, sticky="ew")

In [41]:
ingredients_label = tk.Label(window, text="Ingredientes seleccionados: ")

button_buscar = tk.Button(window, text="Buscar", command=buscar_recetas)

result_label = tk.Label(window, text="Resultados:")

In [42]:
# Crear Treeview para mostrar los resultados
tree = ttk.Treeview(window, columns=("Categoria", "Nombre", "Link_receta"), show='headings')
tree.heading("Categoria", text="Categoria")
tree.heading("Nombre", text="Nombre")
tree.heading("Link_receta", text="Link_receta")
tree.column("Categoria", width=100)
tree.column("Nombre", width=200)
tree.column("Link_receta", width=200)

{}

In [43]:
# Añadir evento de clic al Treeview
tree.bind("<Double-1>", on_treeview_click)

'1931995967488on_treeview_click'

In [44]:
# Posicionar widgets en la ventana usando grid
ingredients_label.grid(row=1, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")
button_buscar.grid(row=2, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")
result_label.grid(row=3, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")
tree.grid(row=4, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="nsew")

In [45]:
# Configurar que las columnas y filas se expandan
for i in range(len(available_ingredients)):
    window.grid_columnconfigure(i, weight=1)
window.grid_rowconfigure(4, weight=1)

In [46]:
# Iniciar el bucle principal de la ventana
window.mainloop()

QUERY: (carne|Pollo)
['carne', 'Pollo']
QUERY: (carne|Pollo),2 Tomate
['carne', 'Pollo', '2Tomate']


# Probando

#Crear ventana principal
window = tk.Tk()
window.title("Aplicación de Recetas")

#Lista para almacenar los ingredientes seleccionados
selected_ingredients = []

#Lista de ingredientes disponibles
available_ingredients = ["Pollo", "Zanahoria", "Papa", "Tomate", "Cebolla","carne"]

# Crear y posicionar los widgets de manera dinámica
for idx, ingredient in enumerate(available_ingredients):
    button = tk.Button(window, text=ingredient, command=lambda ingr=ingredient: add_ingredient(ingr))
    button.grid(row=0, column=idx, padx=5, pady=5, sticky="ew")
ingredients_label = tk.Label(window, text="Ingredientes seleccionados: ")

button_comma = tk.Button(window, text=",", command=lambda: current_separator.set(","))
button_comma.grid(row=0, column=len(available_ingredients), padx=5, pady=5, sticky="ew")

button_pipe = tk.Button(window, text="|", command=lambda: current_separator.set("|"))
button_pipe.grid(row=0, column=len(available_ingredients) + 1, padx=5, pady=5, sticky="ew")

button_buscar = tk.Button(window, text="Buscar", command=buscar_recetas)

result_label = tk.Label(window, text="Resultados:")

# Crear Treeview para mostrar los resultados
tree = ttk.Treeview(window, columns=("Categoria", "Nombre", "Link_receta"), show='headings')
tree.heading("Categoria", text="Categoria")
tree.heading("Nombre", text="Nombre")
tree.heading("Link_receta", text="Link_receta")
tree.column("Categoria", width=100)
tree.column("Nombre", width=200)
tree.column("Link_receta", width=200)

#Añadir evento de clic al Treeview
tree.bind("<Double-1>", on_treeview_click)

# Posicionar widgets en la ventana usando grid
ingredients_label.grid(row=1, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")
button_buscar.grid(row=2, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")
result_label.grid(row=3, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")
tree.grid(row=4, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="nsew")

# Configurar que las columnas y filas se expandan
for i in range(len(available_ingredients)):
    window.grid_columnconfigure(i, weight=1)
window.grid_rowconfigure(4, weight=1)

#Iniciar el bucle principal de la ventana
window.mainloop()